# Idee:
hier werden alle PLZ aus SPARQL-Abfrage und über diese iterieren, damit für jede Gemeinde die entsprechenden Angebote von energiefranken.ch abgerufen und in einer Liste gespeichert werden.


In [50]:
import pandas as pd

df = pd.read_csv('grundlagen_gemeinde_kantone.csv')

# PLZ verwenden und entferne NaN-Werte
plz_series = df['PLZ'].dropna().astype(int).unique()

# Konvertiere in Gmeinde Liste
gemeinden = plz_series.tolist()

damit die Internetseite energiefranken.ch nicht zu sehr belastet wird, wird dies im Umfang dieses Projektes nicht getan, sondern nur für den Kanton Appenzell die entsprechenden Daten abgerufen.

In [51]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re


gemeinden = [9050, 9413, 9108, 9057]

energiefrangen_angebote = []

for gemeinde in gemeinden:
    driver = webdriver.Chrome()
    driver.get(url=f'https://www.energiefranken.ch/de/{gemeinde}')

    # Warten, damit Seite vollständig geladen wird
    driver.implicitly_wait(10)

    # Seite parsen und auch als string abspeichern, in welchem nach der Internetseite gesucht werden kann. Weblink ist auf der dynamischen Seite (ohne, dass die einzelnen Bereiche aufgeklappt sind) nur im Fliesstext vorhanden.
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    str_soup = str(soup)

    # alle Kategorien auslesen
    categories = soup.find_all('div', class_='subsidy-field')

    # Liste für die Ergebnisse
    results = []

    # Iteriere über jede Kategorie
    for category in categories:
        # Extrahiere den Kategorienamen
        category_name = category.find('h3').text.strip()

        # Subsidies finden in der aktuellen Kategorie
        subsidies = category.find_all('td', class_='subsidy-name-print')

        # Iteriere über jede Subsidy in der aktuellen Kategorie
        for subsidy in subsidies:
            # Massnahmen-Titel extrahieren
            subsidy_name = subsidy.text.strip()
            #Anbieter extrahieren
            subsidy_contributor = subsidy.find_next('td', class_='subsidy-contributor-print').text.strip()

            # Website extrahieren
            text_match = re.search(re.escape(subsidy_name), str_soup) #escape sorgt dafür, dass auch subsidy_name mit Klammern im Namen, im str_soup gefunden werden -> Ausdrücke werden automatisch escaped
            if text_match:
                subsidy_website = re.search(r'http[^,]*', str_soup[text_match.end():])
                website = subsidy_website.group(0) if subsidy_website else None
            else:
                website = None

            # Füge die Informationen zur Ergebnisliste hinzu
            results.append({
                'Gemeinde': gemeinde,
                'category': category_name,
                'Programm_Name': str(subsidy_name),
                'Anbieter': subsidy_contributor,
                'Website': re.sub(r'&quot;', '', str(website))
            })
    # Füge die Ergebnisse der aktuellen Gemeinde zur Gesamtliste hinzu
    energiefrangen_angebote.extend(results)



    driver.quit()

In [53]:
# Erstelle ein DataFrame aus den Ergebnissen
df_energiefranken = pd.DataFrame(energiefrangen_angebote)

# Speichere das DataFrame in eine CSV-Datei
df_energiefranken.to_csv('energiefranken_angebote.csv', index=False)

In [54]:
df_energiefranken

,Gemeinde,category,Programm_Name,Anbieter,Website
0,9050,Beratung,Beratung Solarenergie,Kanton Appenzell Innerrhoden,https://energie-ar-ai.ch/cms/index.php/angebot...
1,9050,Beratung,Energieberatung,Kanton Appenzell Innerrhoden,https://energie-ar-ai.ch/cms/index.php/angebot...
2,9050,Beratung,Impulsberatung «erneuerbar heizen»,Bundesamt für Energie,https://erneuerbarheizen.ch/impulsberatung/
3,9050,Heizung,Anschluss an ein Wärmenetz,Kanton Appenzell Innerrhoden,https://www.ai.ch/themen/planen-und-bauen/ener...
4,9050,Heizung,Automatische Holzfeuerung ab 70 kW,Kanton Appenzell Innerrhoden,https://www.ai.ch/themen/planen-und-bauen/ener...
...,...,...,...,...,...
115,9057,"Geräte, Beleuchtung",SALVALUCE Plus: Beleuchtungserneuerung von Woh...,Savenergy Light Solutions GmbH,https://www.salvaluce.ch/
116,9057,"Geräte, Beleuchtung",smart-dry.ch: Effiziente Entfeuchtungsgeräte,smart-dry.ch,http://www.smart-dry.ch
117,9057,Lüftung,Installation Wohnungslüftung,Kanton Appenzell Innerrhoden,https://www.ai.ch/themen/planen-und-bauen/ener...
118,9057,Lüftung,Optivent: Effizientere Lüftungen,Energie Zukunft Schweiz AG,http://www.energiezukunftschweiz.ch/de/themen/...
